#  정보 검색 평가지표(HitRate, MRR, NDCG)

- 정보 검색 시스템의 주요 평가지표 이해
- Hit Rate, MRR, MAP, NDCG의 계산 방법 및 의미 파악
- 최신 Python 라이브러리를 활용한 실습
- RAG 시스템 평가에 적용

---

## 📚 1. 기본 개념 및 배경

### 1.1 정보 검색 평가의 중요성

정보 검색(Information Retrieval) 시스템의 성능을 평가하는 것은 다음과 같은 이유로 중요합니다:

- **사용자 만족도**: 관련성 높은 문서를 상위에 배치하여 사용자 경험 향상
- **시스템 개선**: 정량적 지표를 통한 객관적 성능 비교
- **비즈니스 가치**: 검색 품질 향상을 통한 클릭률, 전환율 개선

### 1.2 평가 지표의 분류

#### 순위 인식 여부 (Rank-Aware vs Rank-Unaware)
- **Rank-Unaware**: 검색 결과의 순서를 고려하지 않음 (Precision, Recall)
- **Rank-Aware**: 검색 결과의 순서를 고려함 (MRR, MAP, NDCG)

#### 평가 방식
- **이진 관련성**: 관련 있음(1) 또는 없음(0)
- **등급 관련성**: 다단계 점수 (1~5점 등)

---

## 🔧 2. 환경 설정

### 2.1 라이브러리 임포트



In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# 기본 라이브러리
import numpy as np
import pandas as pd
from typing import List, Dict, Tuple
import matplotlib.pyplot as plt
import seaborn as sns

# LangChain (RAG 시스템용)
from langchain_core.documents import Document

### 2.2 최신 평가 라이브러리 소개

- **ranx**: 고속 평가 라이브러리 (2022~)
    - Numba 기반 고성능 구현
    - 통계적 유의성 검정 지원
    - LaTeX 테이블 자동 생성

- **pytrec_eval**: 표준 평가 도구
    - TREC 공식 평가 도구의 Python 인터페이스
    - 학술 연구에서 널리 사용

- **ir-measures**: 정보 검색 평가 라이브러리
    - 다양한 평가 지표 지원
    - 사용자 정의 지표 생성 가능

- **설치 방법**:

    ```python
    # 평가 전용 라이브러리
    !pip install ranx-k 
    ```

---

## 📊 3. 샘플 데이터 준비

### 3.1 검색 시나리오 설정

고객 서비스 문의 검색 시스템을 평가하는 상황을 가정합니다.

In [3]:
from langchain_core.documents import Document
from textwrap import dedent

# 문서 데이터베이스 (전체 10개 문서로 확장)
document_pool = [
    Document(
        page_content="배송 지연 문의 - 주문 상품의 배송이 예상보다 늦어지고 있습니다.",
        metadata={"id": "doc1", "category": "배송", "priority": "높음"}
    ),
    Document(
        page_content="결제 오류 - 카드 결제 시 오류가 발생하여 주문이 완료되지 않았습니다.",
        metadata={"id": "doc2", "category": "결제", "priority": "높음"}
    ),
    Document(
        page_content="제품 교환 - 사이즈가 맞지 않아 다른 사이즈로 교환하고 싶습니다.",
        metadata={"id": "doc3", "category": "교환/반품", "priority": "중간"}
    ),
    Document(
        page_content="주문 취소 및 환불 - 주문을 취소하고 전액 환불받고 싶습니다.",
        metadata={"id": "doc4", "category": "취소/환불", "priority": "중간"}
    ),
    Document(
        page_content="포인트 적립 문의 - 구매 후 포인트가 적립되지 않았습니다.",
        metadata={"id": "doc5", "category": "포인트", "priority": "낮음"}
    ),
    Document(
        page_content="재고 문의 - 원하는 상품이 품절인데 언제 재입고되나요?",
        metadata={"id": "doc6", "category": "재고", "priority": "중간"}
    ),
    Document(
        page_content="회원가입 문의 - 회원가입 시 인증번호가 오지 않습니다.",
        metadata={"id": "doc7", "category": "회원", "priority": "낮음"}
    ),
    Document(
        page_content="쿠폰 사용 문의 - 보유한 쿠폰이 적용되지 않습니다.",
        metadata={"id": "doc8", "category": "쿠폰", "priority": "낮음"}
    ),
    Document(
        page_content="배송 주소 변경 - 배송 전에 주소를 변경하고 싶습니다.",
        metadata={"id": "doc9", "category": "배송", "priority": "중간"}
    ),
    Document(
        page_content="상품 리뷰 문의 - 구매한 상품에 대한 리뷰 작성 방법을 알고 싶습니다.",
        metadata={"id": "doc10", "category": "리뷰", "priority": "낮음"}
    )
]

# 검색 쿼리와 정답 (현실적인 시나리오)
queries_and_ground_truth = [
    {
        "query": "배송 늦어요",
        "relevant_docs": ["doc1", "doc9"]  # 배송 관련 문서들
    },
    {
        "query": "결제가 안 되고 포인트도 문제가 있어요", 
        "relevant_docs": ["doc2", "doc5"]  # 결제 + 포인트 관련
    },
    {
        "query": "주문 취소하고 싶어요",
        "relevant_docs": ["doc4"]  # 취소 관련만
    },
    {
        "query": "교환 환불 정책이 궁금해요",
        "relevant_docs": ["doc3", "doc4"]  # 교환 + 환불 관련
    },
    {
        "query": "쿠폰이 왜 안 써져요?",
        "relevant_docs": ["doc8"]  # 쿠폰 관련만
    }
]

# 시스템 검색 결과 (현실적인 성능 차이 반영)
system_results = [
    # Query 1: "배송 늦어요" - 좋은 성능 (정답 1, 2위)
    ["doc1", "doc9", "doc6", "doc2", "doc7"],
    
    # Query 2: "결제가 안 되고 포인트도 문제가 있어요" - 보통 성능 (정답이 2, 4위)
    ["doc7", "doc2", "doc3", "doc5", "doc1"],
    
    # Query 3: "주문 취소하고 싶어요" - 나쁜 성능 (정답이 5위)
    ["doc3", "doc6", "doc2", "doc1", "doc4"],
    
    # Query 4: "교환 환불 정책이 궁금해요" - 혼재된 성능 (정답이 1, 6위)
    ["doc3", "doc7", "doc5", "doc8", "doc2", "doc4"],
    
    # Query 5: "쿠폰이 왜 안 써져요?" - 매우 나쁜 성능 (정답이 없음!)
    ["doc5", "doc2", "doc7", "doc1", "doc10"]
]

print("검색 시나리오 설정 완료!")
print(f"총 문서 수: {len(document_pool)}")
print(f"평가 쿼리 수: {len(queries_and_ground_truth)}")

검색 시나리오 설정 완료!
총 문서 수: 10
평가 쿼리 수: 5


---

## 📈 4. 평가 지표 상세 설명

### 4.1 Hit Rate (적중률)

#### 개념
**Hit Rate@k**는 상위 k개 검색 결과에 관련 문서가 **하나라도** 포함되어 있는지 측정하는 지표입니다.

#### 특징
- **이진 평가**: 있음(1) 또는 없음(0)
- **순서 무관**: 관련 문서의 위치는 고려하지 않음
- **직관적**: 가장 이해하기 쉬운 지표

#### 계산 공식

```markdown
Hit Rate@k = (적중한 쿼리 수) / (전체 쿼리 수)

각 쿼리의 적중 여부 = 1 if 상위 k개에 관련 문서 존재 else 0
```

In [5]:
def calculate_hit_rate(ground_truth: List[List[str]], 
                      predictions: List[List[str]], 
                      k: int) -> float:
    """
    Hit Rate@k 계산
    """
    hits = 0
    total_queries = len(ground_truth)
    
    print(f"=== Hit Rate@{k} 계산 과정 ===")
    
    for i, (gt, pred) in enumerate(zip(ground_truth, predictions)):
        # 상위 k개 예측 결과
        top_k_pred = pred[:k]
        
        # 관련 문서가 하나라도 있으면 적중
        hit = any(doc in gt for doc in top_k_pred)
        if hit:
            hits += 1
        
        print(f"Query {i+1}: GT={gt}, Pred@{k}={top_k_pred}")
        print(f"         Hit: {'O' if hit else 'X'}")
    
    hit_rate = hits / total_queries
    print(f"\n총 적중: {hits}/{total_queries} = {hit_rate:.3f}")
    return hit_rate

# 실제 계산
ground_truth = [query["relevant_docs"] for query in queries_and_ground_truth]

print("=== Hit Rate 계산 결과 ===")
for k in [1, 3, 5]:
    hit_rate = calculate_hit_rate(ground_truth, system_results, k)
    print(f"Hit Rate@{k}: {hit_rate:.3f}\n")

=== Hit Rate 계산 결과 ===
=== Hit Rate@1 계산 과정 ===
Query 1: GT=['doc1', 'doc9'], Pred@1=['doc1']
         Hit: O
Query 2: GT=['doc2', 'doc5'], Pred@1=['doc7']
         Hit: X
Query 3: GT=['doc4'], Pred@1=['doc3']
         Hit: X
Query 4: GT=['doc3', 'doc4'], Pred@1=['doc3']
         Hit: O
Query 5: GT=['doc8'], Pred@1=['doc5']
         Hit: X

총 적중: 2/5 = 0.400
Hit Rate@1: 0.400

=== Hit Rate@3 계산 과정 ===
Query 1: GT=['doc1', 'doc9'], Pred@3=['doc1', 'doc9', 'doc6']
         Hit: O
Query 2: GT=['doc2', 'doc5'], Pred@3=['doc7', 'doc2', 'doc3']
         Hit: O
Query 3: GT=['doc4'], Pred@3=['doc3', 'doc6', 'doc2']
         Hit: X
Query 4: GT=['doc3', 'doc4'], Pred@3=['doc3', 'doc7', 'doc5']
         Hit: O
Query 5: GT=['doc8'], Pred@3=['doc5', 'doc2', 'doc7']
         Hit: X

총 적중: 3/5 = 0.600
Hit Rate@3: 0.600

=== Hit Rate@5 계산 과정 ===
Query 1: GT=['doc1', 'doc9'], Pred@5=['doc1', 'doc9', 'doc6', 'doc2', 'doc7']
         Hit: O
Query 2: GT=['doc2', 'doc5'], Pred@5=['doc7', 'doc2', 'doc3', 'd

**예상 출력 분석:**
```markdown
Hit Rate@1: 0.400  # Query 1, 4만 1위에 정답 (5개 중 2개)
Hit Rate@3: 0.600  # Query 1, 2, 4만 상위 3개에 정답 (5개 중 3개)  
Hit Rate@5: 0.800  # Query 5만 상위 5개에도 정답 없음 (5개 중 4개)
```

### 4.2 MRR (Mean Reciprocal Rank)

#### 개념
**MRR**은 **첫 번째 관련 문서의 순위**에 기반한 평가 지표입니다. 사용자가 원하는 정보를 얼마나 빨리 찾을 수 있는지를 측정합니다.

#### 특징
- **순위 고려**: 상위에 있을수록 높은 점수
- **첫 번째만**: 첫 관련 문서 이후는 무시
- **사용자 경험**: 실제 검색 행동과 유사

#### 계산 공식
```markdown
MRR = (1/Q) × Σ(1/rank_i)

여기서:
- Q: 전체 쿼리 수
- rank_i: i번째 쿼리에서 첫 번째 관련 문서의 순위
- 관련 문서가 없으면 0

In [5]:
def calculate_mrr(ground_truth: List[List[str]], 
                  predictions: List[List[str]], 
                  k: int = None) -> float:
    """
    Mean Reciprocal Rank 계산
    """
    reciprocal_ranks = []
    
    print(f"=== MRR 계산 과정 ===")
    
    for i, (gt, pred) in enumerate(zip(ground_truth, predictions)):
        # 상위 k개만 고려 (k가 None이면 전체)
        search_list = pred[:k] if k else pred
        
        # 첫 번째 관련 문서의 순위 찾기
        first_relevant_rank = None
        for rank, doc_id in enumerate(search_list, 1):
            if doc_id in gt:
                first_relevant_rank = rank
                break
        
        # Reciprocal Rank 계산
        rr = 1.0 / first_relevant_rank if first_relevant_rank else 0.0
        reciprocal_ranks.append(rr)
        
        print(f"Query {i+1}: GT={gt}")
        print(f"         Pred={search_list[:5]}")
        print(f"         첫 관련문서 순위: {first_relevant_rank}, RR: {rr:.3f}")
    
    mrr = sum(reciprocal_ranks) / len(reciprocal_ranks)
    print(f"\n평균 RR: {sum(reciprocal_ranks):.3f} / {len(reciprocal_ranks)} = {mrr:.3f}")
    return mrr

# 실제 계산
print("=== MRR 계산 결과 ===")
mrr_score = calculate_mrr(ground_truth, system_results, k=10)
print(f"최종 MRR: {mrr_score:.3f}")

=== MRR 계산 결과 ===
=== MRR 계산 과정 ===
Query 1: GT=['doc1', 'doc9']
         Pred=['doc1', 'doc9', 'doc6', 'doc2', 'doc7']
         첫 관련문서 순위: 1, RR: 1.000
Query 2: GT=['doc2', 'doc5']
         Pred=['doc7', 'doc2', 'doc3', 'doc5', 'doc1']
         첫 관련문서 순위: 2, RR: 0.500
Query 3: GT=['doc4']
         Pred=['doc3', 'doc6', 'doc2', 'doc1', 'doc4']
         첫 관련문서 순위: 5, RR: 0.200
Query 4: GT=['doc3', 'doc4']
         Pred=['doc3', 'doc7', 'doc5', 'doc8', 'doc2']
         첫 관련문서 순위: 1, RR: 1.000
Query 5: GT=['doc8']
         Pred=['doc5', 'doc2', 'doc7', 'doc1', 'doc10']
         첫 관련문서 순위: None, RR: 0.000

평균 RR: 2.700 / 5 = 0.540
최종 MRR: 0.540


**예상 출력 분석:**
```markdown
Query 1: 첫 관련문서 순위: 1, RR: 1.000  # doc1이 1위
Query 2: 첫 관련문서 순위: 2, RR: 0.500  # doc2가 2위  
Query 3: 첫 관련문서 순위: 5, RR: 0.200  # doc4가 5위
Query 4: 첫 관련문서 순위: 1, RR: 1.000  # doc3이 1위
Query 5: 첫 관련문서 순위: None, RR: 0.000  # 관련문서 없음

최종 MRR: (1.000 + 0.500 + 0.200 + 1.000 + 0.000) / 5 = 0.540
```

### 4.3 MAP (Mean Average Precision)

#### 개념
**MAP@k**는 상위 k개 결과에서 **모든 관련 문서의 정확도를 종합적으로 평가**하는 지표입니다.

#### 특징
- **순위 고려**: 상위에 있는 관련 문서에 더 높은 가중치
- **전체 고려**: 모든 관련 문서의 위치를 반영
- **정확도 중심**: 정밀도(Precision)의 평균

#### 계산 공식
```markdown
MAP@k = (1/Q) × Σ AP@k_i

AP@k = (1/R) × Σ P(j) × rel(j)

여기서:
- Q: 전체 쿼리 수
- R: 관련 문서 수
- P(j): j번째 위치에서의 정밀도
- rel(j): j번째 문서가 관련 있으면 1, 아니면 0

In [6]:
def calculate_map_at_k(ground_truth: List[List[str]], 
                       predictions: List[List[str]], 
                       k: int) -> float:
    """
    Mean Average Precision@k 계산
    """
    average_precisions = []
    
    print(f"=== MAP@{k} 계산 과정 ===")
    
    for i, (gt, pred) in enumerate(zip(ground_truth, predictions)):
        # 상위 k개만 고려
        top_k_pred = pred[:k]
        
        # Average Precision 계산
        relevant_count = 0
        precision_sum = 0.0
        
        print(f"\nQuery {i+1}: GT={gt}, Pred={top_k_pred}")
        
        for rank, doc_id in enumerate(top_k_pred, 1):
            if doc_id in gt:
                relevant_count += 1
                precision_at_rank = relevant_count / rank
                precision_sum += precision_at_rank
                print(f"  위치 {rank}: {doc_id} (관련O) - P@{rank} = {precision_at_rank:.3f}")
            else:
                print(f"  위치 {rank}: {doc_id} (관련X)")
        
        # AP 계산: 관련 문서 수로 나누어 정규화
        ap = precision_sum / len(gt) if len(gt) > 0 else 0.0
        average_precisions.append(ap)
        print(f"  AP@{k} = {precision_sum:.3f} / {len(gt)} = {ap:.3f}")
    
    map_score = sum(average_precisions) / len(average_precisions)
    print(f"\n최종 MAP@{k}: {sum(average_precisions):.3f} / {len(average_precisions)} = {map_score:.3f}")
    return map_score

# 실제 계산
print("=== MAP@k 계산 결과 ===")
for k in [3, 5]:
    map_score = calculate_map_at_k(ground_truth, system_results, k)
    print(f"MAP@{k}: {map_score:.3f}\n")

=== MAP@k 계산 결과 ===
=== MAP@3 계산 과정 ===

Query 1: GT=['doc1', 'doc9'], Pred=['doc1', 'doc9', 'doc6']
  위치 1: doc1 (관련O) - P@1 = 1.000
  위치 2: doc9 (관련O) - P@2 = 1.000
  위치 3: doc6 (관련X)
  AP@3 = 2.000 / 2 = 1.000

Query 2: GT=['doc2', 'doc5'], Pred=['doc7', 'doc2', 'doc3']
  위치 1: doc7 (관련X)
  위치 2: doc2 (관련O) - P@2 = 0.500
  위치 3: doc3 (관련X)
  AP@3 = 0.500 / 2 = 0.250

Query 3: GT=['doc4'], Pred=['doc3', 'doc6', 'doc2']
  위치 1: doc3 (관련X)
  위치 2: doc6 (관련X)
  위치 3: doc2 (관련X)
  AP@3 = 0.000 / 1 = 0.000

Query 4: GT=['doc3', 'doc4'], Pred=['doc3', 'doc7', 'doc5']
  위치 1: doc3 (관련O) - P@1 = 1.000
  위치 2: doc7 (관련X)
  위치 3: doc5 (관련X)
  AP@3 = 1.000 / 2 = 0.500

Query 5: GT=['doc8'], Pred=['doc5', 'doc2', 'doc7']
  위치 1: doc5 (관련X)
  위치 2: doc2 (관련X)
  위치 3: doc7 (관련X)
  AP@3 = 0.000 / 1 = 0.000

최종 MAP@3: 1.750 / 5 = 0.350
MAP@3: 0.350

=== MAP@5 계산 과정 ===

Query 1: GT=['doc1', 'doc9'], Pred=['doc1', 'doc9', 'doc6', 'doc2', 'doc7']
  위치 1: doc1 (관련O) - P@1 = 1.000
  위치 2: doc9 (관련O) - P

**예상 출력 분석:**
```markdown
Query 1: AP@3 = (1.000 + 1.000) / 2 = 1.000  # 1,2위 모두 정답
Query 2: AP@3 = (0.500) / 2 = 0.250           # 2위만 정답
Query 3: AP@3 = 0.000 / 1 = 0.000             # 3위 안에 정답 없음  
Query 4: AP@3 = (1.000) / 2 = 0.500           # 1위만 정답
Query 5: AP@3 = 0.000 / 1 = 0.000             # 3위 안에 정답 없음

MAP@3: (1.000 + 0.250 + 0.000 + 0.500 + 0.000) / 5 = 0.350
```

### 4.4 NDCG (Normalized Discounted Cumulative Gain)

#### 개념
**NDCG@k**는 관련성 점수와 순위를 모두 고려한 **가장 정교한 평가 지표**입니다.

#### 특징
- **등급 관련성**: 다단계 관련성 점수 지원
- **위치 할인**: 하위 순위일수록 가중치 감소
- **정규화**: 이상적인 순위와 비교하여 0~1 범위

#### 계산 공식
```markdown
NDCG@k = DCG@k / IDCG@k

DCG@k = Σ(i=1 to k) (2^rel_i - 1) / log₂(i + 1)

여기서:
- rel_i: i번째 문서의 관련성 점수
- IDCG@k: 이상적인 순위에서의 DCG   
```

In [8]:
import math

def calculate_ndcg_at_k(ground_truth: List[List[str]], 
                        predictions: List[List[str]], 
                        k: int,
                        relevance_scores: Dict[str, Dict[str, int]] = None) -> float:
    """
    NDCG@k 계산 (이진 관련성 또는 등급 관련성)
    """
    if relevance_scores is None:
        # 이진 관련성: 관련 있으면 1, 없으면 0
        relevance_scores = {}
        for i, gt in enumerate(ground_truth):
            query_id = f"query_{i}"
            relevance_scores[query_id] = {doc: 1 for doc in gt}
    
    ndcg_scores = []
    
    print(f"=== NDCG@{k} 계산 과정 ===")
    
    for i, (gt, pred) in enumerate(zip(ground_truth, predictions)):
        query_id = f"query_{i}"
        top_k_pred = pred[:k]
        
        print(f"\nQuery {i+1}: GT={gt}, Pred={top_k_pred}")
        
        # DCG 계산
        dcg = 0.0
        for rank, doc_id in enumerate(top_k_pred, 1):
            rel_score = relevance_scores.get(query_id, {}).get(doc_id, 0)
            if rel_score > 0:
                gain = (2**rel_score - 1) / math.log2(rank + 1)
                dcg += gain
                print(f"  위치 {rank}: {doc_id} (관련성={rel_score}) - Gain: {gain:.3f}")
            else:
                print(f"  위치 {rank}: {doc_id} (관련성=0)")
        
        # IDCG 계산 (이상적인 순위)
        ideal_scores = sorted(relevance_scores.get(query_id, {}).values(), 
                             reverse=True)[:k]
        idcg = 0.0
        for rank, rel_score in enumerate(ideal_scores, 1):
            if rel_score > 0:
                idcg += (2**rel_score - 1) / math.log2(rank + 1)
        
        # NDCG 계산
        ndcg = dcg / idcg if idcg > 0 else 0.0
        ndcg_scores.append(ndcg)
        
        print(f"  DCG: {dcg:.3f}, IDCG: {idcg:.3f}, NDCG: {ndcg:.3f}")
    
    final_ndcg = sum(ndcg_scores) / len(ndcg_scores)
    print(f"\n최종 NDCG@{k}: {sum(ndcg_scores):.3f} / {len(ndcg_scores)} = {final_ndcg:.3f}")
    return final_ndcg

# 실제 계산 (이진 관련성)
print("=== NDCG@k 계산 결과 ===")
for k in [3, 5]:
    ndcg_score = calculate_ndcg_at_k(ground_truth, system_results, k)
    print(f"NDCG@{k}: {ndcg_score:.3f}\n")

=== NDCG@k 계산 결과 ===
=== NDCG@3 계산 과정 ===

Query 1: GT=['doc1', 'doc9'], Pred=['doc1', 'doc9', 'doc6']
  위치 1: doc1 (관련성=1) - Gain: 1.000
  위치 2: doc9 (관련성=1) - Gain: 0.631
  위치 3: doc6 (관련성=0)
  DCG: 1.631, IDCG: 1.631, NDCG: 1.000

Query 2: GT=['doc2', 'doc5'], Pred=['doc7', 'doc2', 'doc3']
  위치 1: doc7 (관련성=0)
  위치 2: doc2 (관련성=1) - Gain: 0.631
  위치 3: doc3 (관련성=0)
  DCG: 0.631, IDCG: 1.631, NDCG: 0.387

Query 3: GT=['doc4'], Pred=['doc3', 'doc6', 'doc2']
  위치 1: doc3 (관련성=0)
  위치 2: doc6 (관련성=0)
  위치 3: doc2 (관련성=0)
  DCG: 0.000, IDCG: 1.000, NDCG: 0.000

Query 4: GT=['doc3', 'doc4'], Pred=['doc3', 'doc7', 'doc5']
  위치 1: doc3 (관련성=1) - Gain: 1.000
  위치 2: doc7 (관련성=0)
  위치 3: doc5 (관련성=0)
  DCG: 1.000, IDCG: 1.631, NDCG: 0.613

Query 5: GT=['doc8'], Pred=['doc5', 'doc2', 'doc7']
  위치 1: doc5 (관련성=0)
  위치 2: doc2 (관련성=0)
  위치 3: doc7 (관련성=0)
  DCG: 0.000, IDCG: 1.000, NDCG: 0.000

최종 NDCG@3: 2.000 / 5 = 0.400
NDCG@3: 0.400

=== NDCG@5 계산 과정 ===

Query 1: GT=['doc1', 'doc9'], Pred=[

---

## 📊 5. 지표별 특성 비교 분석

- **Hit Rate**: 검색 시스템의 기본 성능 확인
- **MRR**: 사용자가 빠르게 답을 찾는 것이 중요한 경우
- **MAP**: 모든 관련 문서의 순위가 중요한 경우
- **NDCG**: 등급별 관련성과 순위 모두 고려해야 하는 경우

In [9]:
def comprehensive_evaluation():
    """모든 지표를 한번에 계산하여 특성 비교"""
    
    results = {}
    
    print("=" * 60)
    print("        종합 평가 결과 비교")
    print("=" * 60)
    
    # Hit Rate 계산
    for k in [1, 3, 5]:
        hr = calculate_hit_rate(ground_truth, system_results, k)
        results[f"Hit_Rate@{k}"] = hr
    
    print()
    
    # MRR 계산
    mrr = calculate_mrr(ground_truth, system_results)
    results["MRR"] = mrr
    
    print()
    
    # MAP 계산  
    for k in [3, 5]:
        map_score = calculate_map_at_k(ground_truth, system_results, k)
        results[f"MAP@{k}"] = map_score
    
    print()
    
    # NDCG 계산
    for k in [3, 5]:
        ndcg = calculate_ndcg_at_k(ground_truth, system_results, k)
        results[f"NDCG@{k}"] = ndcg
    
    # 결과 요약 테이블
    print("\n" + "=" * 60)
    print("            최종 결과 요약")
    print("=" * 60)
    print(f"{'지표':<12} {'점수':<8} {'해석'}")
    print("-" * 50)
    
    interpretations = {
        "Hit_Rate@1": "첫 번째 결과의 유용성",
        "Hit_Rate@3": "상위 3개 결과의 포괄성", 
        "Hit_Rate@5": "상위 5개 결과의 포괄성",
        "MRR": "첫 관련문서 발견 속도",
        "MAP@3": "상위 3개의 정확도", 
        "MAP@5": "상위 5개의 정확도",
        "NDCG@3": "상위 3개의 종합 품질",
        "NDCG@5": "상위 5개의 종합 품질"
    }
    
    for metric, score in results.items():
        interp = interpretations.get(metric, "")
        print(f"{metric:<12} {score:<8.3f} {interp}")
    
    return results

# 종합 평가 실행
evaluation_results = comprehensive_evaluation()

        종합 평가 결과 비교
=== Hit Rate@1 계산 과정 ===
Query 1: GT=['doc1', 'doc9'], Pred@1=['doc1']
         Hit: O
Query 2: GT=['doc2', 'doc5'], Pred@1=['doc7']
         Hit: X
Query 3: GT=['doc4'], Pred@1=['doc3']
         Hit: X
Query 4: GT=['doc3', 'doc4'], Pred@1=['doc3']
         Hit: O
Query 5: GT=['doc8'], Pred@1=['doc5']
         Hit: X

총 적중: 2/5 = 0.400
=== Hit Rate@3 계산 과정 ===
Query 1: GT=['doc1', 'doc9'], Pred@3=['doc1', 'doc9', 'doc6']
         Hit: O
Query 2: GT=['doc2', 'doc5'], Pred@3=['doc7', 'doc2', 'doc3']
         Hit: O
Query 3: GT=['doc4'], Pred@3=['doc3', 'doc6', 'doc2']
         Hit: X
Query 4: GT=['doc3', 'doc4'], Pred@3=['doc3', 'doc7', 'doc5']
         Hit: O
Query 5: GT=['doc8'], Pred@3=['doc5', 'doc2', 'doc7']
         Hit: X

총 적중: 3/5 = 0.600
=== Hit Rate@5 계산 과정 ===
Query 1: GT=['doc1', 'doc9'], Pred@5=['doc1', 'doc9', 'doc6', 'doc2', 'doc7']
         Hit: O
Query 2: GT=['doc2', 'doc5'], Pred@5=['doc7', 'doc2', 'doc3', 'doc5', 'doc1']
         Hit: O
Query 3: GT

---

## 🛠️ 6. 최신 라이브러리 활용

### 6.1 ranx 라이브러리 사용법
```python
# ranx 설치 및 임포트
!pip install ranx
```

In [ ]:
from ranx import Qrels, Run, evaluate, compare

# 데이터 변환
def convert_to_ranx_format(ground_truth, predictions):
    """데이터를 ranx 형식으로 변환"""
    
    # Qrels (Ground Truth) 형식: {query_id: {doc_id: relevance_score}}
    qrels_dict = {}
    for i, gt_docs in enumerate(ground_truth):
        query_id = f"q_{i+1}"
        qrels_dict[query_id] = {doc_id: 1 for doc_id in gt_docs}
    
    # Run (Predictions) 형식: {query_id: {doc_id: score}}
    run_dict = {}
    for i, pred_docs in enumerate(predictions):
        query_id = f"q_{i+1}"
        # 순위에 따라 점수 부여 (높은 순위 = 높은 점수)
        run_dict[query_id] = {
            doc_id: len(pred_docs) - rank 
            for rank, doc_id in enumerate(pred_docs)
        }
    
    return qrels_dict, run_dict


qrels_dict, run_dict = convert_to_ranx_format(ground_truth, system_results)

# ranx 객체 생성
qrels = Qrels(qrels_dict, name="Customer_Service_Queries")
run = Run(run_dict, name="Search_System_v1")

print("=== ranx를 사용한 평가 ===")

# 여러 지표 동시 계산
metrics = ["hit_rate@1", "hit_rate@3", "hit_rate@5", "mrr", "map@3", "map@5", "ndcg@3", "ndcg@5"]
results = evaluate(qrels, run, metrics)

print("ranx 라이브러리 계산 결과:")
for metric, score in results.items():
    print(f"{metric}: {score:.3f}")

=== ranx를 사용한 평가 ===


c:\Users\User\ktds-llm\.venv\Lib\site-packages\ranx\metrics\hit_rate.py:38: NumbaTypeSafetyWarning: unsafe cast from uint64 to int64. Precision may be lost.
  scores[i] = _hit_rate(qrels[i], run[i], k, rel_lvl)


ranx 라이브러리 계산 결과:
hit_rate@1: 0.400
hit_rate@3: 0.600
hit_rate@5: 0.800
mrr: 0.540
map@3: 0.350
map@5: 0.440
ndcg@3: 0.400
ndcg@5: 0.530


### 6.2 pytrec_eval 사용법
```python
# pytrec_eval 설치 
!pip install pytrec-eval
```

In [ ]:
import pytrec_eval
import json

# pytrec_eval 형식으로 변환
qrel_trec = {}
run_trec = {}

for i, (gt, pred) in enumerate(zip(ground_truth, system_results)):
    query_id = f"q{i+1}"
    
    # qrel 형식: {query_id: {doc_id: relevance}}
    qrel_trec[query_id] = {doc_id: 1 for doc_id in gt}
    
    # run 형식: {query_id: {doc_id: score}}
    run_trec[query_id] = {
        doc_id: 1.0 - (rank * 0.1) 
        for rank, doc_id in enumerate(pred)
    }

# 평가자 생성
evaluator = pytrec_eval.RelevanceEvaluator(
    qrel_trec, 
    {'map', 'ndcg', 'recip_rank', 'P_1', 'P_3'} # 평가 지표 설정 (예: MAP, NDCG, Reciprocal Rank, Precision@1, Precision@3)
)

# 평가 실행
results = evaluator.evaluate(run_trec)

print("=== pytrec_eval 결과 ===")
print(json.dumps(results, indent=2))

# 평균 점수 계산 (k=5)
metrics_avg = {}
for metric in ['map', 'ndcg', 'recip_rank', 'P_1', 'P_3']:
    scores = [results[qid][metric] for qid in results.keys()]
    metrics_avg[metric] = sum(scores) / len(scores)
    print(f"{metric}: {metrics_avg[metric]:.3f}")

---

## 📝 7. 실습 문제

**문제 1**: 다음 검색 결과에 대해 Hit Rate@3, MRR, MAP@3을 계산하세요.


- **예상 결과:**
    ```markdown
    Hit Rate@3: 0.750 (4개 중 3개 쿼리에서 관련 문서 발견)
    MRR: 0.458 ((1/1 + 1/2 + 0/1 + 1/3) / 4)
    MAP@3: 0.417 (정답 위치와 개수에 따라 계산)
    ```


In [ ]:
# 연습 데이터 (다양한 성능 수준)
practice_ground_truth = [
    ["A", "B"],         # Query 1의 정답
    ["C"],              # Query 2의 정답  
    ["D", "E"],         # Query 3의 정답
    ["F"]               # Query 4의 정답
]

practice_predictions = [
    ["A", "X", "B"],      # Query 1: 좋은 성능 (1,3위)
    ["Y", "C", "Z"],      # Query 2: 보통 성능 (2위)
    ["W", "V", "U"],      # Query 3: 나쁜 성능 (정답 없음)
    ["T", "S", "F"]       # Query 4: 나쁜 성능 (3위)
]

# 여기에 코드를 작성하세요

In [ ]:
def solve_practice_problem():
    """연습 문제 해답 함수"""
    print("=== 연습 문제 풀이 ===")
    
    # Hit Rate@3 계산
    hit_rate = calculate_hit_rate(practice_ground_truth, practice_predictions, 3)
    print(f"Hit Rate@3: {hit_rate:.3f}")
    
    # MRR 계산  
    mrr = calculate_mrr(practice_ground_truth, practice_predictions)
    print(f"MRR: {mrr:.3f}")
    
    # MAP@3 계산
    map_score = calculate_map_at_k(practice_ground_truth, practice_predictions, 3)
    print(f"MAP@3: {map_score:.3f}")

# 정답 확인
solve_practice_problem()

**문제 2**: 검색 시스템을 개선했을 때의 성능 변화를 측정해보세요.

In [ ]:
# 개선 전 vs 개선 후 시스템 비교
system_v1_results = system_results  # 기존 시스템

# 개선된 시스템 (v2) - 더 나은 성능
system_v2_results = [
    ["doc1", "doc9", "doc2", "doc6", "doc7"],  # Query 1: 더 좋아짐
    ["doc2", "doc5", "doc3", "doc7", "doc1"],  # Query 2: 개선됨  
    ["doc4", "doc3", "doc6", "doc2", "doc1"],  # Query 3: 크게 개선됨
    ["doc3", "doc4", "doc7", "doc5", "doc8"],  # Query 4: 개선됨
    ["doc8", "doc5", "doc2", "doc7", "doc1"]   # Query 5: 개선됨
]

# 여기에 코드를 작성하세요

In [ ]:
def compare_systems():
    """두 시스템의 성능 비교"""
    print("=== 시스템 성능 비교 ===")
    
    metrics = ["Hit Rate@3", "MRR", "MAP@3", "NDCG@3"]
    
    # v1 성능
    v1_hr3 = calculate_hit_rate(ground_truth, system_v1_results, 3)
    v1_mrr = calculate_mrr(ground_truth, system_v1_results)
    v1_map3 = calculate_map_at_k(ground_truth, system_v1_results, 3)
    v1_ndcg3 = calculate_ndcg_at_k(ground_truth, system_v1_results, 3)
    
    # v2 성능  
    v2_hr3 = calculate_hit_rate(ground_truth, system_v2_results, 3)
    v2_mrr = calculate_mrr(ground_truth, system_v2_results)
    v2_map3 = calculate_map_at_k(ground_truth, system_v2_results, 3)
    v2_ndcg3 = calculate_ndcg_at_k(ground_truth, system_v2_results, 3)
    
    # 결과 비교
    v1_scores = [v1_hr3, v1_mrr, v1_map3, v1_ndcg3]
    v2_scores = [v2_hr3, v2_mrr, v2_map3, v2_ndcg3]
    
    print("\n시스템 성능 비교:")
    print(f"{'지표':<12} {'v1':<8} {'v2':<8} {'개선율':<8}")
    print("-" * 40)
    
    for metric, v1, v2 in zip(metrics, v1_scores, v2_scores):
        improvement = ((v2 - v1) / v1 * 100) if v1 > 0 else float('inf')
        print(f"{metric:<12} {v1:<8.3f} {v2:<8.3f} {improvement:<8.1f}%")

# 시스템 비교 실행
compare_systems()

---

## 🎓 8. 핵심 개념 정리

### 8.1 지표별 특성 요약

| 지표 | 강점 | 약점 | 적합한 상황 |
|------|------|------|------------|
| **Hit Rate** | 직관적, 계산 간단 | 순위 무시, 이진적 | 기본 성능 확인 |
| **MRR** | 사용자 경험 반영 | 첫 번째만 고려 | QA, 단일 정답 검색 |
| **MAP** | 모든 관련문서 고려 | 등급 관련성 미지원 | 포괄적 검색 평가 |
| **NDCG** | 가장 정교한 평가 | 복잡함, 등급 필요 | 추천 시스템, 웹검색 |

### 8.2 시스템별 추천 지표

| 시스템 유형 | 주요 지표 | 이유 |
|------------|-----------|------|
| **웹 검색 엔진** | NDCG@10, MAP@10, MRR | 다양한 관련성 수준과 순위가 중요 |
| **상품 추천** | Hit Rate@5, NDCG@5, MAP@5 | 상위 추천의 정확성이 핵심 |
| **문서 검색** | MRR, MAP@10, Hit Rate@3 | 정확한 문서 발견이 우선 |
| **FAQ 검색** | MRR, Hit Rate@1, MAP@3 | 빠른 정답 찾기가 중요 |
| **이커머스 검색** | NDCG@3, Hit Rate@10, MRR | 상품 관련성과 순위 모두 중요 |

### 8.3 성능 해석 가이드라인

| 지표 | 우수 | 양호 | 보통 | 개선 필요 |
|------|------|------|------|-----------|
| **Hit Rate** | ≥ 0.800 | 0.600 - 0.799 | 0.400 - 0.599 | < 0.400 |
| **MRR** | ≥ 0.700 | 0.500 - 0.699 | 0.300 - 0.499 | < 0.300 |
| **MAP** | ≥ 0.600 | 0.400 - 0.599 | 0.200 - 0.399 | < 0.200 |
| **NDCG** | ≥ 0.700 | 0.500 - 0.699 | 0.300 - 0.499 | < 0.300 |


### 8.4 성능 개선 가이드

- 🔴 개선 필요 (성능 향상 우선순위)
    1. **데이터 품질 개선**: 문서 전처리, 중복 제거
    2. **임베딩 모델 변경**: 더 적합한 도메인 특화 모델 사용
    3. **검색 파라미터 조정**: top-k, 임계값 등

- 🟡 보통 (최적화 고려)
    1. **하이브리드 검색 도입**: 키워드 + 벡터 검색 결합
    2. **리랭킹 시스템 추가**: 재순위화 알고리즘 적용
    3. **쿼리 확장**: 동의어, 관련어 추가

- 🟢 양호/우수 (미세 조정)
    1. **파인튜닝**: 도메인 특화 데이터로 모델 조정
    2. **앙상블 방법**: 여러 검색 전략 조합
    3. **사용자 피드백 반영**: 클릭률, 만족도 데이터 활용


---

## 📚 9. 추가 학습 자료

### 9.1 관련 논문
1. **"Evaluation Measures for Information Retrieval"** - Christopher Manning
2. **"Normalized Discounted Cumulative Gain"** - Kalervo Järvelin
3. **"A Comparison of Statistical Significance Tests"** - Mark Smucker

### 9.2 라이브러리
- **ranx**: 고성능 평가 라이브러리
- **pytrec_eval**: 표준 TREC 평가 도구
- **ir_measures**: 통합 평가 메트릭 라이브러리
- **PyTerrier**: 정보 검색 실험 플랫폼

### 9.3 실무 적용
1. **A/B 테스트와 연계**: 오프라인 지표와 온라인 성과의 상관관계 분석
2. **비즈니스 지표 연결**: 클릭률, 전환율과 IR 지표의 관계 파악
3. **지속적 모니터링**: 실시간 성능 추적 시스템 구축

---